In [ ]:
import pandas as pd
import s3fs
from smart_open import open
import boto3
from io import StringIO # python3; python2: BytesIO 
from boto3.s3.transfer import TransferConfig
import metrics
import torch
from transformers import *
import numpy as np
import ast
import time

In [ ]:
user_tokens = pd.read_csv('s3://recsyschallenge2020/user_tokens.csv')
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
model = BertModel.from_pretrained('/dev/bert/')

In [ ]:
def calculate_average(row):
    sum_tensors = torch.zeros([768], dtype=torch.float32)
    tweet_token_list = ast.literal_eval(row)
    for token_list in tweet_token_list:
        token_list_embeddings = model(torch.tensor(list(map(int, token_list.split('\t')))).unsqueeze(0))[0][0][0]
        sum_tensors = sum_tensors + token_list_embeddings
    avg = sum_tensors/len(row)
    return avg

In [ ]:
user_tokens['avg_embeddings'] = user_tokens['text_ tokens'].apply(lambda x: calculate_average(x))

In [ ]:
user_tokens.to_csv('s3://recsyschallenge2020/user_tokens_embeddins.csv', index = False)